In [3]:
###############################################################################################################
# Audio-driven upper-body motion synthesis on a humanoid robot
# Computer Science Tripos Part III Project
# Jan Ondras (jo356@cam.ac.uk), Trinity College, University of Cambridge
# 2017/18
###############################################################################################################
# Comparison of LSTM architecture capacities
###############################################################################################################

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Masking, TimeDistributed, Dropout, Bidirectional
from tabulate import tabulate

N_LSTM_units_range = [1,2,3,4,5,6,7,8,9, 10,11,12,13,14, 15, 16,17,18, 19, 20, 25, 30, 35, 40, 45, 50, 55] # 6.6 hours, 13 features

feature_sets = [
    'MFCC 13',
    'LogFB 26', 
    'LogFB 52', 
    'LogFB 78', 
    'BLSTM LogFB 26'
]

params = np.zeros( (len(N_LSTM_units_range), len(feature_sets)) )

SEGMENT_LEN = 300
N_targets = 11

for FS_i, FS in enumerate(feature_sets):
    for units_i, N_LSTM_units in enumerate(N_LSTM_units_range):
        
        N_features = int(FS[-2:])
        #print N_features

        # Create LSTM model
        model = Sequential()
        model.add( Masking(mask_value=0., input_shape=(SEGMENT_LEN, N_features)) )
        if FS_i == 4: # BLSTM
            model.add( Bidirectional(LSTM(N_LSTM_units, return_sequences=True)) )
        else:
            model.add( LSTM(N_LSTM_units, return_sequences=True) ) 
        model.add( TimeDistributed(Dense(N_targets, activation='sigmoid')) )
        #model.compile(loss='mean_squared_error', optimizer='adam')
        #print model.summary()

        params[units_i, FS_i] = model.count_params()

        
tab_data = params
headers = ['# LSTM units'] + feature_sets
print tabulate(tab_data, headers=headers, 
               tablefmt='fancy_grid', 
               showindex=N_LSTM_units_range, 
               numalign='center'
              )

╒════════════════╤═══════════╤════════════╤════════════╤════════════╤══════════════════╕
│  # LSTM units  │  MFCC 13  │  LogFB 26  │  LogFB 52  │  LogFB 78  │  BLSTM LogFB 26  │
╞════════════════╪═══════════╪════════════╪════════════╪════════════╪══════════════════╡
│       1        │    82     │    134     │    238     │    342     │       257        │
├────────────────┼───────────┼────────────┼────────────┼────────────┼──────────────────┤
│       2        │    161    │    265     │    473     │    681     │       519        │
├────────────────┼───────────┼────────────┼────────────┼────────────┼──────────────────┤
│       3        │    248    │    404     │    716     │    1028    │       797        │
├────────────────┼───────────┼────────────┼────────────┼────────────┼──────────────────┤
│       4        │    343    │    551     │    967     │    1383    │       1091       │
├────────────────┼───────────┼────────────┼────────────┼────────────┼──────────────────┤
│       5        │   